In [1]:
import numpy as np
import random as r
from builtins import input

In [2]:
class Agent:
    def __init__(self, epsilon=0.1, lr=0.5):
        self.state_history = []
        self.eps = epsilon 
        self.verbose = False
        self.lr = lr
        self.foo = False
        
    def setFoo(self):
        self.foo = True
        
    def setV(self, env):
        self.V = -np.ones(env.num_states)
            
    def set_one_value(self, state, winner, ended):
        if ended:
            if winner == self.sym:
                self.V[state] = 1
            else:
                self.V[state] = 0
        else:
            self.V[state] = 0.5
                 

    def set_symbol(self, symbol):
        self.sym = symbol
        
    # after one episode clean it up for the new game
    def reset_history(self):
        self.state_history = []
   
    # use epsilon-greedy
    def take_action(self, env):
        rand = r.random()
        rand_row = 0
        rand_column = 0

        if rand > self.eps:
            best_value = -1
            for i in range(3):
                for j in range(3):
                    if env.is_empty(i, j):
                        env.board[i][j] = self.sym
                        state = env.get_state()
                        value = self.V[state] 
                        env.board[i][j] = 0
                        if best_value < value:
                            best_value = value
                            rand_row = i
                            rand_column = j        
        else:
            if self.foo:
                print("Take random action")
            bool = True
            while bool:
                rand_row = r.randint(0, 2)
                rand_column = r.randint(0, 2)
                if env.is_empty(rand_row, rand_column):
                    bool = False
                    
        env.board[rand_row][rand_column] = self.sym
        
    
    # which move in which order
    def update_state_history(self, state):
        self.state_history.append(state)
    
    # at the end of an episode
    # backtrack over states
    def update(self, env):
        
        new_V = self.V
        V_next = env.reward(self.sym) 
        
        for state in reversed(self.state_history):
            V_current = self.V[state] + self.lr * (V_next - self.V[state])
            new_V[state] = V_current
            V_next = V_current # because we go one step back
            
        self.V = new_V
        self.reset_history()

In [3]:
class Human:
    def __init__(self):
        self.V = -np.ones(3**9)
    
    def set_symbol(self, symbol):
        self.sym = symbol

    def take_action(self, env):
        while True:
            i, j = input("\nEnter coordinates i,j:\n").split(',')
            i = int(i)
            j = int(j)
            if env.is_empty(i,j):
                env.board[i,j] = self.sym
                break
            print("Field is set!")
            print()
    
    def update(self, env):
        pass
            
    def set_one_value(self, state, winner, ended):
        pass
    
    def update_state_history(self, state):
        pass

In [4]:
class Environment:
    def __init__(self):
        self.width = 3
        self.height = 3
        
        self.board = np.zeros((self.width, self.height))

        self.winner = None # saves X (1) or O (-1) 
        self.ended = False
        
        self.X = 1
        self.O = -1
        
        self.num_states = 3**(self.width*self.height) 
    
    def reward(self, sym):
        if not self.game_over(): 
            return 0
        if self.winner == sym:
            return 1
        else:
            return 0
    
    def is_empty(self, i, j):
        if self.board[i][j] == 0:
            return True
        return False
    
    def reset_board(self):
        self.board = np.zeros((self.width, self.height))

    # state = 3^(N-1) * b(N-1) + ... + 3^(1) * b(1) + 3^(0) * b(0)
    # b ∈ {0,1,2} -> 0 (empty), 1 (X), 2 (O) 
    # all possible states will be covered (also the ones who cannot occur)
    def get_state(self):
        hash = 0
        k = 0
        
        for i in range(self.width):
            for j in range(self.height):
                if self.is_empty(i,j):
                    v = 0
                elif self.board[i][j] == self.X: 
                    v = 1
                elif self.board[i][j] == self.O:
                    v = 2   
                hash += 3**k * v
                k += 1
                
        return hash
    
    def game_over(self, force_recalculate=False):
        
        count = 0
        # check all rows
        for i in range(3):
            for j in range(self.width):
                if self.board[i][j] == env.X:
                    count += 1
            if count == 3:
                self.winner = env.X
                self.ended = True
                return True
            count = 0

        count = 0
        for i in range(3):
            for j in range(self.width):
                if self.board[i][j] == env.O:
                    count += 1
            if count == 3:
                self.winner = env.O
                self.ended = True
                return True
            count = 0
            
        count = 0
        # check all columns
        for i in range(3):
            for j in range(self.height):
                if self.board[j][i] == env.X:
                    count += 1
            if count == 3:
                self.winner = env.X
                self.ended = True
                return True
            count = 0
            
        count = 0
        for i in range(3):
            for j in range(self.height):
                if self.board[j][i] == env.O:
                    count += 1
            if count == 3:
                self.winner = env.O
                self.ended = True
                return True
            count = 0
            
        count = 0
        # check diagonals
        diag = np.diag(self.board)

        for i in range(3):
            if diag[i] == env.X:
                count += 1
        if count == 3:
            self.winner = env.X
            self.ended = True
            return True
        
        count = 0
        for i in range(3):
            if diag[i] == env.O:
                count += 1
        if count == 3:
            self.winner = env.O
            self.ended = True
            return True

        count = 0
        diag = np.array([self.board[0][2], self.board[1][1], self.board[2][0]])

        for i in range(3):
            if diag[i] == env.X:
                count += 1
        if count == 3:
            self.winner = env.X
            self.ended = True
            return True
        
        count = 0
        for i in range(3):
            if diag[i] == env.O:
                count += 1
        if count == 3:
            self.winner = env.O
            self.ended = True
            return True
        count = 0

        # check if draw 
        if np.all((self.board == 0) == False):
            self.winner = None
            self.ended = True
            return True
            
        self.winner = None
        return False
    
    def is_draw(self):
        return self.ended and self.winner is None
    
    def draw_board(self):
        # change later
        for i in range(self.width):
            print("--------------")
            for j in range(self.height):
                if self.board[i][j] == 1:
                    print(" X  ", end="")
                elif self.board[i][j] == -1:
                    print(" O  ", end="")
                else:
                    print("    ", end="")
                if j != 2:
                    print("|", end="")
            print()
        print("--------------")

        
        
        

In [5]:
def play_game(p1, p2, env, draw=1):
    
    current_player = None
    
    while not env.game_over():
        if current_player == p1:
            current_player = p2
        else:
            current_player = p1
                
        if current_player == p2 and draw == 2:
            env.draw_board()
    
        current_player.take_action(env)
        initialize_value_func(p1,p2,env)
        
        state = env.get_state()
        
        p1.update_state_history(state)
        p2.update_state_history(state)
        
    # do the value function update after one episode
    p1.update(env)
    p2.update(env)

In [6]:
def initialize_value_func(p1,p2,env):
    state = env.get_state()
    
    if env.winner == 1:
        winner_p1 = 1
        winner_p2 = 0
        ended = True
    elif env.winner == -1:
        winner_p1 = 0
        winner_p2 = 1
        ended = True
    else:
        winner_p1, winner_p2 = 0.5, 0.5
        ended = False
        
    if p1.V[state] == -1:
        p1.set_one_value(state, winner_p1, ended)
    if p2.V[state] == -1:
        p2.set_one_value(state, winner_p2, ended)
    return

In [ ]:
if __name__ == "__main__":
    p1 = Agent()
    p2 = Agent()
    
    env = Environment()

    p1.set_symbol(env.X)
    p2.set_symbol(env.O)
    
    p1.setV(env)
    p2.setV(env)
    
    p1.set_symbol(env.X)
    p2.set_symbol(env.O)
        
    for i in range(10000):
        if i % 200 == 0:
            print(i)
        play_game(p1, p2, Environment()) 
    print()
    
    human = Human()
    human.set_symbol(env.O)
    
    # human vs. agent
    p1.setFoo()
    while True:
        env = Environment()
        play_game(p1, human, env, draw=2)
        env.draw_board()
        answer = input("\nPlay again? [y/n]")
        if answer == 'n':
            break

0
200
400
600
800
1000
1200
1400
1600
1800
2000
2200
2400
2600
2800
3000
3200
3400
3600
3800
4000
4200
4400
4600
4800
5000
5200
5400
5600
5800
6000
6200
6400
6600
6800
7000
7200
7400
7600
7800
8000
8200
8400
8600
8800
9000
9200
9400
9600
9800

--------------
    |    |    
--------------
    | X  |    
--------------
    |    |    
--------------

Enter coordinates i,j:
2,2
--------------
    |    |    
--------------
    | X  |    
--------------
    | X  | O  
--------------

Enter coordinates i,j:
0,1
--------------
    | O  | X  
--------------
    | X  |    
--------------
    | X  | O  
--------------

Enter coordinates i,j:
2,0
--------------
 X  | O  | X  
--------------
    | X  |    
--------------
 O  | X  | O  
--------------

Enter coordinates i,j:
1,2
--------------
 X  | O  | X  
--------------
 X  | X  | O  
--------------
 O  | X  | O  
--------------

Play again? [y/n]y
--------------
    |    |    
--------------
    | X  |    
--------------
    |    |    
---------